In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [2]:
!ls

1.benshi.ai_submission.ipynb AirQualityUCI.csv
1.benshi_initial_tests.ipynb AirQualityUCI2.csv
2.VAR_air_prediction.ipynb


In [3]:
#read the data
df = pd.read_csv("AirQualityUCI2.csv", sep=";", parse_dates=[['Date', 'Time']])
df.columns

Index(['Date_Time', 'CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)',
       'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
       'PT08.S5(O3)', 'T', 'RH', 'AH'],
      dtype='object')

In [4]:
df.dtypes

Date_Time        object
CO(GT)           object
PT08.S1(CO)       int64
NMHC(GT)          int64
C6H6(GT)         object
PT08.S2(NMHC)     int64
NOx(GT)           int64
PT08.S3(NOx)      int64
NO2(GT)           int64
PT08.S4(NO2)      int64
PT08.S5(O3)       int64
T                object
RH               object
AH               object
dtype: object

In [5]:
df['CO(GT)']=df['CO(GT)'].apply(lambda x: x.replace(",","."))
df['C6H6(GT)']=df['C6H6(GT)'].apply(lambda x: x.replace(",","."))
df['T']=df['T'].apply(lambda x: x.replace(",","."))
df['RH']=df['RH'].apply(lambda x: x.replace(",","."))
df['AH']=df['AH'].apply(lambda x: x.replace(",","."))


df['CO(GT)'] = df['CO(GT)'].apply(pd.to_numeric)
df['C6H6(GT)'] = df['C6H6(GT)'].apply(pd.to_numeric)
df['T'] = df['T'].apply(pd.to_numeric)
df['RH'] = df['RH'].apply(pd.to_numeric)
df['AH'] = df['AH'].apply(pd.to_numeric)

In [6]:
df.dtypes

Date_Time         object
CO(GT)           float64
PT08.S1(CO)        int64
NMHC(GT)           int64
C6H6(GT)         float64
PT08.S2(NMHC)      int64
NOx(GT)            int64
PT08.S3(NOx)       int64
NO2(GT)            int64
PT08.S4(NO2)       int64
PT08.S5(O3)        int64
T                float64
RH               float64
AH               float64
dtype: object

In [7]:
df.tail()

,Date_Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
9352,04/04/2005 10.00.00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,04/04/2005 11.00.00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,04/04/2005 12.00.00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,04/04/2005 13.00.00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139
9356,04/04/2005 14.00.00,2.2,1071,-200,11.9,1047,265,654,168,1129,816,28.5,13.1,0.5028


In [8]:
df['Date_Time'] = pd.to_datetime(df.Date_Time , format = '%d/%m/%Y %H.%M.%S')
data = df.drop(['Date_Time'], axis=1)
data.index = df.Date_Time

In [9]:
#missing value treatment
cols = data.columns
for j in cols:
    for i in range(0,len(data)):
        if data[j][i] == -200:
            data[j][i] = data[j][i-1]

/Users/rsilvei/.pyenv/versions/3.7.2/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
#checking stationarity
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues
johan_test_temp = data.drop([ 'CO(GT)'], axis=1)
coint_johansen(johan_test_temp,-1,1).eig

array([1.75510896e-01, 1.52389933e-01, 1.15120416e-01, 1.04126281e-01,
       9.29485509e-02, 6.89397159e-02, 5.77070988e-02, 3.43554214e-02,
       3.05980659e-02, 1.18697142e-02, 2.46766099e-03, 7.09584856e-05])

In [33]:
#creating the train and validation set
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

In [34]:
endog_list = train['CO(GT)']
endog_list

Date_Time
2004-03-10 18:00:00    2.6
2004-03-10 19:00:00    2.0
2004-03-10 20:00:00    2.2
2004-03-10 21:00:00    2.2
2004-03-10 22:00:00    1.6
                      ... 
2005-01-16 10:00:00    1.0
2005-01-16 11:00:00    1.0
2005-01-16 12:00:00    0.8
2005-01-16 13:00:00    1.0
2005-01-16 14:00:00    0.8
Name: CO(GT), Length: 7485, dtype: float64

In [35]:
dates_list = train.index
len(dates_list)

7485

In [37]:
exog_list = train[['PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)']]
exog_list

,PT08.S1(CO),NMHC(GT),C6H6(GT)
Date_Time,,,
2004-03-10 18:00:00,1360,150,11.9
2004-03-10 19:00:00,1292,112,9.4
2004-03-10 20:00:00,1402,88,9.0
2004-03-10 21:00:00,1376,80,9.2
2004-03-10 22:00:00,1272,51,6.5
...,...,...,...
2005-01-16 10:00:00,841,275,2.6
2005-01-16 11:00:00,850,275,2.6
2005-01-16 12:00:00,831,275,2.2


In [38]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.arima.model import ARIMA

In [39]:
endog_list.index.inferred_freq

'H'

In [40]:
model = ARIMA(endog=v,exog=exog_list, dates=dates_list)
# model = ARIMA(endog=endog_list, 
#               exog=exog_list, 
#               freq=endog_list.index.inferred_freq)

model_fit = model.fit()

/Users/rsilvei/.pyenv/versions/3.7.2/envs/torch/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


In [41]:
len(valid), len(train)

(1872, 7485)

In [42]:
exog_valid = valid[['PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)']]
exog_valid

,PT08.S1(CO),NMHC(GT),C6H6(GT)
Date_Time,,,
2005-01-16 15:00:00,833,275,2.0
2005-01-16 16:00:00,877,275,2.8
2005-01-16 17:00:00,892,275,3.3
2005-01-16 18:00:00,899,275,3.4
2005-01-16 19:00:00,1008,275,7.1
...,...,...,...
2005-04-04 10:00:00,1314,275,13.5
2005-04-04 11:00:00,1163,275,11.4
2005-04-04 12:00:00,1142,275,12.4


In [43]:
len(valid)

1872

In [48]:
valid['CO(GT)']

Date_Time
2005-01-16 15:00:00    0.7
2005-01-16 16:00:00    1.1
2005-01-16 17:00:00    1.1
2005-01-16 18:00:00    1.2
2005-01-16 19:00:00    2.0
                      ... 
2005-04-04 10:00:00    3.1
2005-04-04 11:00:00    2.4
2005-04-04 12:00:00    2.4
2005-04-04 13:00:00    2.1
2005-04-04 14:00:00    2.2
Name: CO(GT), Length: 1872, dtype: float64

In [45]:
preds = model_fit.forecast(exog=exog_valid, steps=len(valid))
preds

2005-01-16 15:00:00    0.727762
2005-01-16 16:00:00    0.900942
2005-01-16 17:00:00    0.979144
2005-01-16 18:00:00    1.004396
2005-01-16 19:00:00    1.578290
                         ...   
2005-04-04 10:00:00    2.853199
2005-04-04 11:00:00    2.313305
2005-04-04 12:00:00    2.347170
2005-04-04 13:00:00    1.768651
2005-04-04 14:00:00    2.134412
Freq: H, Name: predicted_mean, Length: 1872, dtype: float64

In [49]:
np.sqrt(mean_squared_error(preds, valid['CO(GT)']))

0.7849157216492115

In [50]:
#make final predictions
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

[[2.34596328e+00 1.08633212e+03 2.80762173e+02 1.24130779e+01
  1.05535947e+03 2.80882233e+02 6.59534851e+02 1.68444418e+02
  1.15918056e+03 8.50845529e+02 2.73639014e+01 1.55311062e+01
  5.15317053e-01]]


/Users/rsilvei/.pyenv/versions/3.7.2/envs/torch/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/Users/rsilvei/.pyenv/versions/3.7.2/envs/torch/lib/python3.7/site-packages/statsmodels/base/wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


## Fit in the complete dataset